# Add Board Wrap
* If a piece is on the left edge of the board, add it to the right edge.
* If a piece is on the top edge of the board, add it to the bottom edge.

In [2]:
import itertools
import numpy as np
from IPython.display import clear_output
from sys import exit
from math import floor
from copy import copy

## Utility Function


In [16]:
# check if a value is an integer
def is_int(value):
    try: int(value);   return True
    except: return False

# repalce special characters
def replace_special(s, empty_piece=','):
    return s.replace('-4', ' ').replace('-3', '?').replace('-2', '#').replace('-1', empty_piece)
    

In [20]:
class game(object):
    def __init__(self, w, h, n, init_board=None, board_wrap=False):
        self.player = 0
        self.status = 'active'

        # set up initial board
        if init_board is not None: 
            self.board = init_board
            h, w = init_board.shape
        else: self.board = np.array([[-1]*w]*h)
        self.w, self.h, self.n = w, h, n
        self.col_count = [next((h-k-1 for k in range(h)[::-1] if col[k] >-2), h) for col in self.board.T]
        self.feasible_moves = set(k for k in range(w) if self.col_count[k]<h)
        
        # modify for board wrap
        self.board_wrap = board_wrap
        if board_wrap:
            # add rows to top and bottom
            bot = np.array([self.board[h-1,:]])
            top = np.array([self.board[0,:]])
            self.board = np.concatenate((bot, self.board, top), axis=0)
            
            # add cols to left and right
            left = np.array([self.board[:,0]]).T
            right = np.array([self.board[:,w-1]]).T
            self.board = np.concatenate((right, self.board, left), axis=1)
            
    # extract main board (no margins)
    def main_board(self):
        bw = 1*self.board_wrap
        return self.board[bw:self.h+bw, bw:self.w+bw]
    
        
    # display the status of the board
    def print_board(self):
        if self.board_wrap:
            clear_output() # comment out for debugging
            print('Connect-' + str(self.n))
            
            # extract main board
            mboard = self.main_board()
            
            # print top margin
            lcorner = replace_special(str(self.board[0,0]))
            rcorner = replace_special(str(self.board[0,self.w+1]))
            row = ' '.join(str(r) for r in self.board[0,1:self.w+1])
            row = replace_special(row)
            print(lcorner + '| ' + row + ' |' + rcorner)
            #print(' ' + lcorner + ' ' + row + ' ' + rcorner) # inside

            # extract left and right margins 
            lmargin = [replace_special(str(num)) for num in self.board[1:self.h+1,0]]
            rmargin = [replace_special(str(num)) for num in self.board[1:self.h+1,self.w+1]]
            
            # print left and right margins with actual board
            for k in range(self.h):
                row = ' '.join(str(r) for r in mboard[k,:])
                row = replace_special(row, empty_piece='.')
                print(lmargin[k] + '| ' + row + ' |' + rmargin[k])
                #print('|' + lmargin[k] + ' ' + row + ' ' + rmargin[k] + '|') # inside
                
            # print bottom margin
            lcorner = replace_special(str(self.board[self.h+1,0]))
            rcorner = replace_special(str(self.board[self.h+1,self.w+1]))
            row = ' '.join(str(r) for r in self.board[self.h+1,1:self.w+1])
            row = replace_special(row)
            print(lcorner + '| ' + row + ' |' + rcorner)
            #print(' ' + lcorner + ' ' + row + ' ' + rcorner) # inside
            
            # print column numbers
            print(' ' + '-'*(3+2*self.w))
            # print(' ' + '-'*(3+2*self.w)) # inside
            print('   ' + ' '.join(str(col) for col in range(self.w)))
            

        else:
            clear_output() # comment out for debugging
            print('Connect-' + str(self.n))
            for k in range(self.h):
                row = ' '.join(str(r) for r in self.board[k,:])
                row = replace_special(row, empty_piece='.')
                print('| ' + row + ' |')
            print('-'*(3+2*self.w))
            print(' '.join(str(col) for col in range(self.w)))        

    # get user's input
    def col_input(self):
        # ask for and validate input
        while True:
            col = input('Player ' + str(self.player) + ': Enter the column to place your piece ' + 
                        '(' + str(self.player).replace('1','x') + ') ' )
            if col.lower()=='q': print('Thanks for playing!');   exit(0)
            if col=='`': col = 0 # define an alias for 0

            # check for valid input
            if is_int(col): 
                col = int(col)
                if col>=0 and col<self.w and col in self.feasible_moves: break
        return col

    # count connections to the left and right of pos
    def cnct_gt_n(self, vec, pos): 
        count, player = 1, vec[pos]
        
        # check if at least n in length
        if sum(np.logical_or(vec==player, vec==-3)) < self.n: return False

        for v in vec[:pos][::-1]:
            if v==player or v==-3: count += 1
            else: break
        for v in vec[pos+1:]:
            if v==player or v==-3: count += 1
            else: break
        return count >= self.n

    # extract backward diagonal
    def bdiag(self, board, shift=0):
        h, w = self.board.shape
        if shift==0: return [board[k, w-k-1] for k in range(min(h, w))]
        if shift>0: return [board[k, w-(k+shift)-1] for k in range(min(h, w-shift))]
        return [board[k-shift, w-k-1] for k in range(min(h+shift, w))]   

    # check intersecting row, col, diagonals for connect-n
    def connect_n_check(self, row, col):
        """
        mboard  = self.main_board()
        forward_diag = np.diag(mboard, col-row)
        backward_diag = self.bdiag(mboard, self.w-1-col-row)
        return (self.cnct_gt_n(mboard[:, col], row) or                 # col
                self.cnct_gt_n(mboard[row, :], col) or                 # row
                self.cnct_gt_n(forward_diag, min(row, col)) or         # fdiag
                self.cnct_gt_n(backward_diag, min(row, self.w-1-col))) # bdiag
        """     
        col += 1
        row += 1
        forward_diag = np.diag(self.board, col-row)
        backward_diag = self.bdiag(self.board, self.w+2-1-col-row)
        return (self.cnct_gt_n(self.board[:, col], row) or               # col
                self.cnct_gt_n(self.board[row, :], col) or               # row
                self.cnct_gt_n(forward_diag, min(row, col)) or           # fdiag
                self.cnct_gt_n(backward_diag, min(row, self.w+2-1-col))) # bdiag
    
    # update game        
    def update_game(self, col):
        # update col_count
        self.col_count[col] += 1
        
        # skip special pieces
        row = self.h-self.col_count[col]
        while self.board[row+1, col+1]<-1: 
            row -= 1 
            self.col_count[col] += 1
                    
        # update board
        self.board[row+1, col+1] = self.player # assign move to player
        if row==0: 
            self.board[self.h+1, col+1] = self.player # bot
            if col==self.w-1: self.board[self.h+1, 0] = self.player # bot right
            elif col==0: self.board[self.h+1, self.w+1] = self.player # bot right
                
        if row==self.h-1: 
            self.board[0, col+1] = self.player # top
            if col==self.w-1: self.board[0, 0] = self.player # top right
            elif col==0: self.board[0, self.w+1] = self.player # top right
                
        if col==0: self.board[row+1, self.w+1] = self.player # right
        if col==self.w-1: self.board[row+1, 0] = self.player # left
            
        # update player and feasible moves
        self.player = 1*(self.player==0)   # switch players
        if self.col_count[col]==self.h: self.feasible_moves.remove(col)
                    
        # update status
        if self.connect_n_check(row, col): self.status = 'won';   return
        if not self.feasible_moves: self.status = 'draw'
        
    # two-player game
    def play_game(self):
        while True:
            self.print_board()
            col = self.col_input()
            self.update_game(col)
            if self.status!='active':
                self.print_board()
                if self.status=='won': print('Player ' + str(1-self.player) + ' won!')
                else: print('Game is a draw.')
                return     
w, h, n = 5, 3, 3
init_board = np.array([[-1]*w]*h)


g = game(w, h, n, init_board, board_wrap=True)
g.play_game()

Connect-3
1| 0 0 0 , 1 |0
,| . . . . . |,
1| . . . . 1 |,
1| 0 0 0 . 1 |0
,| , , , , , |,
 -------------
   0 1 2 3 4
Player 0 won!
